In [ ]:
import streamlit as st
from PyPDF2 import PdfReader
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Function to extract text from PDF

def extract_text_from_pdf(file):
    pdf = PdfReader(file)
    text = ""
    for page in pdf.pages:
        text += page.extract_text() or ""
    return text


In [ ]:
# Function to rank resumes based on job description

def rank_resumes(job_description, resumes):
    documents = [job_description] + resumes
    vectorizer = TfidfVectorizer().fit_transform(documents)
    vectors = vectorizer.toarray()
    job_description_vector = vectors[0]
    resume_vectors = vectors[1:]
    cosine_similarities = cosine_similarity([job_description_vector], resume_vectors).flatten()
    return cosine_similarities


In [ ]:
# Streamlit app
st.title("AI Resume Screening & Candidate Ranking System - By PRANAV PATIL")
# Job description input
st.header("Job Description")
job_description = st.text_area("Enter the job description")

In [ ]:
# File uploader
st.header("Upload Resumes")
uploaded_files = st.file_uploader("Upload PDF files", type=["pdf"], accept_multiple_files=True)

In [ ]:
if uploaded_files and job_description:
st.header("Ranking Resumes")
resumes = [extract_text_from_pdf(file) for file in uploaded_files]
scores = rank_resumes(job_description, resumes)
    
# DataFrame to display results
results = pd.DataFrame({"Resume": [file.name for file in uploaded_files], "Score": scores})
results = results.sort_values(by="Score", ascending=False)
 # Show ranking table
st.dataframe(results)


In [ ]:
 # Highlight Resume Score Changes
st.subheader("Resume Score Progress")
    for i, row in results.iterrows():
        st.metric(label=row["Resume"], value=f"{row['Score']:.2f}")

In [ ]:
# Section-wise Score Breakdown (Placeholder Data)
st.subheader("Section-wise Score Breakdown")
categories = ["Skills", "Experience", "Formatting", "Relevance"]
scores = [20, 30, 25, 25]  # Example scores
fig, ax = plt.subplots()
ax.bar(categories, scores, color=['blue', 'green', 'orange', 'red'])
st.pyplot(fig)

In [ ]:
 # Interactive Resume Insights
with st.expander("What You Can Improve"):
st.write("- Add more relevant skills.")
st.write("- Improve formatting consistency.")
st.write("- Highlight key achievements.")

In [ ]:
 # Resume Leaderboard
st.subheader("Resume Leaderboard")
results["Rank"] = range(1, len(results) + 1)
st.dataframe(results.sort_values(by="Score", ascending=False))

In [ ]:
# Save Resume Progress
st.download_button("Download Report", data=results.to_csv(), file_name="resume_ranking_report.csv", mime="text/csv")